# 🚀 Unleashing the Power of Cortex LLMs at JLP: A Mind-Blowing Demo 🎉

Get ready to explore the cutting-edge capabilities of **Cortex Large Language Models (LLMs)** — where next-gen AI meets limitless potential.


# 🛠️ RAW Layer - Build RAW tables from internal Stage

In [ ]:
-- 🎯 Raw Data Tables - The foundation of our data castle
CREATE OR REPLACE TRANSIENT TABLE RAW.CUSTOMER_INTERACTIONS (data VARIANT);
CREATE OR REPLACE TRANSIENT TABLE RAW.PRODUCT_REVIEWS (data VARIANT);
CREATE OR REPLACE TRANSIENT TABLE RAW.SUPPORT_TICKETS (data VARIANT);
CREATE OR REPLACE TRANSIENT TABLE RAW.CUSTOMERS (data VARIANT);

-- 📥 Data Loading - Let's fill these tables with good vibes
COPY INTO RAW.CUSTOMER_INTERACTIONS
FROM @RAW.RAW_DATA_STAGE/customer_interactions.json
FILE_FORMAT = RAW.JSON_FORMAT
ON_ERROR = 'CONTINUE';

COPY INTO RAW.PRODUCT_REVIEWS
FROM @RAW.RAW_DATA_STAGE/product_reviews.json
FILE_FORMAT = RAW.JSON_FORMAT
ON_ERROR = 'CONTINUE';

COPY INTO RAW.SUPPORT_TICKETS
FROM @RAW.RAW_DATA_STAGE/support_tickets.json
FILE_FORMAT = RAW.JSON_FORMAT
ON_ERROR = 'CONTINUE';

COPY INTO RAW.CUSTOMERS
FROM @RAW.RAW_DATA_STAGE/customers.json
FILE_FORMAT = RAW.JSON_FORMAT
ON_ERROR = 'CONTINUE';

# 🧹 CONFORM Layer - Clean and transform that data!

In [ ]:
-- Customer Interactions
CREATE OR REPLACE TABLE ANALYTICS.STG_CUSTOMER_INTERACTIONS AS
SELECT
    data:interaction_id::VARCHAR AS interaction_id,
    data:customer_id::VARCHAR AS customer_id,
    TRY_TO_TIMESTAMP_NTZ(data:interaction_date::VARCHAR) AS interaction_date,
    data:agent_id::VARCHAR AS agent_id,
    data:interaction_type::VARCHAR AS interaction_type,
    data:interaction_notes::VARCHAR AS interaction_notes
FROM RAW.CUSTOMER_INTERACTIONS;

-- Product Reviews
CREATE OR REPLACE TABLE ANALYTICS.STG_PRODUCT_REVIEWS AS
SELECT
    data:review_id::VARCHAR AS review_id,
    data:customer_id::VARCHAR AS customer_id,
    data:product_id::VARCHAR AS product_id,
    TRY_TO_TIMESTAMP_NTZ(data:review_date::VARCHAR) AS review_date,
    data:review_rating::NUMBER AS review_rating,
    data:review_text::VARCHAR AS review_text,
    data:review_language::VARCHAR AS review_language
FROM RAW.PRODUCT_REVIEWS;

-- Support Tickets
CREATE OR REPLACE TABLE ANALYTICS.STG_SUPPORT_TICKETS AS
SELECT
    data:ticket_id::VARCHAR AS ticket_id,
    data:customer_id::VARCHAR AS customer_id,
    TRY_TO_TIMESTAMP_NTZ(data:ticket_date::VARCHAR) AS ticket_date,
    data:ticket_status::VARCHAR AS ticket_status,
    data:ticket_category::VARCHAR AS ticket_category,
    data:ticket_description::VARCHAR AS ticket_description
FROM RAW.SUPPORT_TICKETS;

-- Customer Dimension
CREATE OR REPLACE TABLE ANALYTICS.CUSTOMER_BASE AS
SELECT
    data:customer_id::VARCHAR AS customer_id,
    data:persona::VARCHAR AS persona,
    TRY_TO_DATE(data:sign_up_date::VARCHAR) AS sign_up_date,
    data:products_owned::NUMBER AS products_owned,
    data:lifetime_value::NUMBER AS lifetime_value
FROM RAW.CUSTOMERS;

# 📊 CONSUME Layer -  Fact Tables (where the real insights live)

In [ ]:
-- Customer Interactions Fact
CREATE OR REPLACE TABLE ANALYTICS.FACT_CUSTOMER_INTERACTIONS AS
SELECT
    i.interaction_id,
    i.customer_id,
    i.interaction_date,
    i.agent_id,
    i.interaction_type,
    i.interaction_notes,
    -- Add sentiment analysis
    SNOWFLAKE.CORTEX.SENTIMENT(i.interaction_notes) AS sentiment_score
FROM ANALYTICS.STG_CUSTOMER_INTERACTIONS i
WHERE i.interaction_notes IS NOT NULL;

In [ ]:
-- Product Reviews Fact
CREATE OR REPLACE TABLE ANALYTICS.FACT_PRODUCT_REVIEWS AS
SELECT
    r.review_id,
    r.customer_id,
    r.product_id,
    r.review_date,
    r.review_rating,
    r.review_text,
    r.review_language,
    -- Add sentiment analysis
    SNOWFLAKE.CORTEX.SENTIMENT(r.review_text) AS sentiment_score,
    -- Add standardized text
    CASE
        WHEN CONTAINS(LOWER(r.review_language), 'german') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'de', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'french') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'fr', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'spanish') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'es', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'italian') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'it', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'portuguese') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'pt', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'chinese') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'zh', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'japanese') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'ja', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'korean') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'ko', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'russian') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'ru', 'en')
        WHEN CONTAINS(LOWER(r.review_language), 'arabic') THEN SNOWFLAKE.CORTEX.TRANSLATE(r.review_text, 'ar', 'en')
        ELSE r.review_text
    END AS review_text_english
FROM ANALYTICS.STG_PRODUCT_REVIEWS r
WHERE r.review_text IS NOT NULL LIMIT 10;

In [ ]:
-- Support Tickets Fact
CREATE OR REPLACE TABLE ANALYTICS.FACT_SUPPORT_TICKETS AS
SELECT
    t.ticket_id,
    t.customer_id,
    t.ticket_date,
    t.ticket_status,
    t.ticket_category,
    t.ticket_description,
    -- Add sentiment analysis
    SNOWFLAKE.CORTEX.SENTIMENT(t.ticket_description) AS sentiment_score,
    -- Add priority classification
    SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
        t.ticket_description,
        [
            {
                'label': 'Critical',
                'description': 'Requires immediate attention and resolution',
                'examples': [
                    'System is down and customers cannot place orders',
                    'Security breach detected',
                    'Payment processing completely stopped',
                    'All users locked out of the system',
                    'Cannot access my account',
                    'Order stuck in processing',
                    'Payment failed',
                    'Website not loading',
                    'Critical feature not working',
                    'Data loss or corruption'
                ]
            },
            {
                'label': 'High',
                'description': 'Should be resolved within 24 hours',
                'examples': [
                    'Customer cannot complete checkout',
                    'Order status stuck in processing',
                    'Unable to access account',
                    'Payment failed for multiple customers',
                    'Product not working as expected',
                    'Account access issues',
                    'Billing problems',
                    'Service disruption',
                    'Performance issues',
                    'Security concerns'
                ]
            },
            {
                'label': 'Medium',
                'description': 'Should be resolved within 3 days',
                'examples': [
                    'Product image not displaying correctly',
                    'Slow response times in certain areas',
                    'Minor UI issues',
                    'Account settings not saving',
                    'Feature not working as expected',
                    'Minor display issues',
                    'Non-critical performance problems',
                    'General functionality questions',
                    'Minor account issues',
                    'Non-urgent technical problems'
                ]
            },
            {
                'label': 'Low',
                'description': 'Can be handled in regular queue',
                'examples': [
                    'General product questions',
                    'Feature request',
                    'UI enhancement suggestion',
                    'Documentation clarification',
                    'How-to questions',
                    'Product information request',
                    'General feedback',
                    'Non-technical questions',
                    'Account information request',
                    'General support questions'
                ]
            }
        ],
        {
            'task_description': 'Classify the urgency level of this support ticket based on its description. When in doubt, classify as Critical or High priority if the issue affects core functionality or user access.'
        }
    )['label'] AS priority_level,
    -- Add customer expectations
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-3-5-sonnet',
        'What specific timeframe or deadline does the customer mention or expect for resolution? ' || t.ticket_description
    ) AS expected_resolution_timeframe,
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-3-5-sonnet',
        'What compensation, refund, or specific remedy is the customer seeking? ' || t.ticket_description
    ) AS requested_remedy
FROM ANALYTICS.STG_SUPPORT_TICKETS t
WHERE t.ticket_description IS NOT NULL LIMIT 10;

# 🔍 CONSUME Layer - Time to dig deep!

In [ ]:
-- Sentiment Analysis
CREATE OR REPLACE TABLE ANALYTICS.SENTIMENT_ANALYSIS AS
WITH interaction_sentiment AS (
    SELECT
        customer_id,
        interaction_date,
        sentiment_score,
        'interaction' AS source_type
    FROM ANALYTICS.FACT_CUSTOMER_INTERACTIONS
),
review_sentiment AS (
    SELECT
        customer_id,
        review_date AS interaction_date,
        sentiment_score,
        'review' AS source_type
    FROM ANALYTICS.FACT_PRODUCT_REVIEWS
),
ticket_sentiment AS (
    SELECT
        customer_id,
        ticket_date AS interaction_date,
        sentiment_score,
        'ticket' AS source_type
    FROM ANALYTICS.FACT_SUPPORT_TICKETS
)
SELECT * FROM interaction_sentiment
UNION ALL
SELECT * FROM review_sentiment
UNION ALL
SELECT * FROM ticket_sentiment;

In [ ]:
-- Sentiment Trends Analysis
CREATE OR REPLACE TABLE ANALYTICS.SENTIMENT_TRENDS AS
WITH sentiment_data AS (
    SELECT
        customer_id,
        sentiment_score,
        interaction_date,
        FIRST_VALUE(sentiment_score) OVER (PARTITION BY customer_id ORDER BY interaction_date) AS first_sentiment,
        LAST_VALUE(sentiment_score) OVER (PARTITION BY customer_id ORDER BY interaction_date 
                                         ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_sentiment
    FROM ANALYTICS.SENTIMENT_ANALYSIS
)
SELECT
    customer_id,
    ARRAY_AGG(sentiment_score) WITHIN GROUP (ORDER BY interaction_date) AS sentiment_history,
    AVG(sentiment_score) AS avg_sentiment,
    MIN(sentiment_score) AS min_sentiment,
    MAX(sentiment_score) AS max_sentiment,
    MAX(sentiment_score) - MIN(sentiment_score) AS sentiment_volatility,
    CASE
        WHEN COUNT(*) > 1 THEN 
            MAX(last_sentiment) - MAX(first_sentiment)
        ELSE 0
    END AS sentiment_trend
FROM sentiment_data
GROUP BY customer_id;

In [ ]:
-- Ticket Pattern Analysis
CREATE OR REPLACE TABLE ANALYTICS.TICKET_PATTERNS AS
SELECT
    customer_id,
    COUNT(*) AS ticket_count,
    MIN(ticket_date) AS first_ticket_date,
    MAX(ticket_date) AS most_recent_ticket_date,
    ARRAY_AGG(ticket_category) WITHIN GROUP (ORDER BY ticket_date) AS ticket_categories,
    ARRAY_AGG(priority_level) WITHIN GROUP (ORDER BY ticket_date) AS ticket_priorities
FROM ANALYTICS.FACT_SUPPORT_TICKETS
GROUP BY customer_id;

In [ ]:
-- Customer Insight Summaries
CREATE OR REPLACE TABLE ANALYTICS.INSIGHT_SUMMARIES AS
SELECT
    customer_id,
    SNOWFLAKE.CORTEX.COMPLETE(
        'claude-3-5-sonnet',
        [
            {
                'role': 'user',
                'content': 'Summarize the following customer interactions in 100 words or less: ' || 
                ARRAY_TO_STRING(
                    ARRAY_AGG(
                        CASE
                            WHEN interaction_notes IS NOT NULL THEN interaction_notes
                            WHEN review_text IS NOT NULL THEN review_text
                            WHEN ticket_description IS NOT NULL THEN ticket_description
                        END
                    ),
                    ' | '
                )
            }
        ],
        {
            'max_tokens': 100
        }
    ) AS customer_summary
FROM (
    SELECT customer_id, interaction_notes, NULL as review_text, NULL as ticket_description
    FROM ANALYTICS.FACT_CUSTOMER_INTERACTIONS
    UNION ALL
    SELECT customer_id, NULL, review_text, NULL
    FROM ANALYTICS.FACT_PRODUCT_REVIEWS
    UNION ALL
    SELECT customer_id, NULL, NULL, ticket_description
    FROM ANALYTICS.FACT_SUPPORT_TICKETS LIMIT 10
)
GROUP BY customer_id;

In [ ]:
-- 👥 Customer Persona Analysis - Get to know your peeps
CREATE OR REPLACE TABLE ANALYTICS.CUSTOMER_PERSONA_SIGNALS AS
SELECT 
    cb.customer_id,
    
    -- Sentiment analysis
    st.avg_sentiment,
    st.sentiment_trend,
    st.sentiment_volatility,
    CASE 
        WHEN st.avg_sentiment < -0.3 THEN 'Negative'
        WHEN st.avg_sentiment > 0.3 THEN 'Positive'
        ELSE 'Neutral'
    END AS overall_sentiment,
    
    -- Support history
    tpat.ticket_count,
    tpat.ticket_categories,
    tpat.ticket_priorities,
    
    -- Review ratings
    AVG(pr.review_rating) AS avg_rating,
    
    -- Customer summary
    is_summary.customer_summary,
    
    -- Customer persona
    CASE
        WHEN st.avg_sentiment > 0.5 AND COALESCE(tpat.ticket_count, 0) <= 1 THEN 'Satisfied'
        WHEN st.avg_sentiment < -0.3 AND COALESCE(tpat.ticket_count, 0) >= 3 THEN 'Frustrated'
        WHEN st.sentiment_volatility > 0.7 THEN 'Mixed'
        WHEN st.sentiment_trend > 0.3 THEN 'Improving'
        WHEN st.sentiment_trend < -0.3 THEN 'Deteriorating'
        ELSE 'Neutral'
    END AS derived_persona,
    
    -- Churn risk
    CASE
        WHEN (st.avg_sentiment < -0.3 AND COALESCE(tpat.ticket_count, 0) >= 1) 
             OR (st.sentiment_trend < -0.2 AND COALESCE(tpat.ticket_count, 0) >= 1)
             OR (st.avg_sentiment < -0.2 AND st.sentiment_trend < -0.1) THEN 'High'
        WHEN (st.avg_sentiment < -0.1 AND st.sentiment_trend < 0) 
             OR (COALESCE(tpat.ticket_count, 0) >= 2) THEN 'Medium'
        ELSE 'Low'
    END AS churn_risk,
    
    -- Upsell opportunity
    CASE
        WHEN st.avg_sentiment > 0.3 AND COALESCE(tpat.ticket_count, 0) <= 1 THEN 'High'
        WHEN st.sentiment_trend > 0.3 THEN 'Medium'
        ELSE 'Low'
    END AS upsell_opportunity
FROM ANALYTICS.CUSTOMER_BASE cb
LEFT JOIN ANALYTICS.SENTIMENT_TRENDS st USING (customer_id)
LEFT JOIN ANALYTICS.INSIGHT_SUMMARIES is_summary USING (customer_id)
LEFT JOIN ANALYTICS.TICKET_PATTERNS tpat USING (customer_id)
LEFT JOIN ANALYTICS.FACT_PRODUCT_REVIEWS pr USING (customer_id)
GROUP BY 
    cb.customer_id, 
    st.avg_sentiment, 
    st.sentiment_trend, 
    st.sentiment_volatility,
    tpat.ticket_count,
    tpat.ticket_categories,
    tpat.ticket_priorities,
    is_summary.customer_summary;

CREATE OR REPLACE DATABASE DBT_CORTEX_LLMS CLONE CLONE_DBT_CORTEX_LLMS_1000;


# 🎯 Time to Get Some Sweet Insights!

In [ ]:
-- 1. 📊 Customer Vibes Overview - How's everyone feeling?
SELECT 
    overall_sentiment,
    COUNT(*) as customer_count,
    ROUND(AVG(avg_sentiment), 2) as avg_sentiment_score,
    ROUND(AVG(avg_rating), 2) as avg_product_rating
FROM ANALYTICS.CUSTOMER_PERSONA_SIGNALS
GROUP BY overall_sentiment
ORDER BY customer_count DESC;

In [ ]:
-- 2. 🚨 Churn Risk Check - Who's thinking about ghosting us?
SELECT 
    churn_risk,
    COUNT(*) as customer_count,
    ROUND(AVG(avg_sentiment), 2) as avg_sentiment_score,
    ROUND(AVG(sentiment_trend), 2) as avg_sentiment_trend,
    SUM(ticket_count) as total_tickets,
    ROUND(AVG(ticket_count), 1) as avg_tickets_per_customer,
    COUNT(CASE WHEN overall_sentiment = 'Negative' THEN 1 END) as negative_sentiment_count,
    COUNT(CASE WHEN overall_sentiment = 'Positive' THEN 1 END) as positive_sentiment_count
FROM ANALYTICS.CUSTOMER_PERSONA_SIGNALS
GROUP BY churn_risk
ORDER BY total_tickets DESC, avg_sentiment_score ASC
LIMIT 10;

In [ ]:
-- 3. 🎫 Support Ticket Heat Map - What's hot in the support queue?
SELECT 
    priority_level,
    COUNT(*) as ticket_count,
    ROUND(AVG(sentiment_score), 2) as avg_sentiment,
    COUNT(DISTINCT customer_id) as unique_customers
FROM ANALYTICS.FACT_SUPPORT_TICKETS
GROUP BY priority_level
ORDER BY 
    CASE priority_level
        WHEN 'Critical' THEN 1
        WHEN 'High' THEN 2
        WHEN 'Medium' THEN 3
        WHEN 'Low' THEN 4
    END;

In [ ]:
-- 4. 🗺️ Customer Journey Map - Plot those customer adventures
WITH customer_journey AS (
    SELECT 
        cps.customer_id,
        cps.derived_persona,
        cps.churn_risk,
        cps.upsell_opportunity,
        cps.avg_sentiment,
        cps.sentiment_trend,
        cps.ticket_count,
        cps.avg_rating,
        cps.customer_summary
    FROM ANALYTICS.CUSTOMER_PERSONA_SIGNALS cps
)
SELECT 
    derived_persona,
    churn_risk,
    upsell_opportunity,
    COUNT(*) as customer_count,
    ROUND(AVG(avg_sentiment), 2) as avg_sentiment,
    ROUND(AVG(avg_rating), 2) as avg_rating,
    ROUND(AVG(ticket_count), 2) as avg_tickets
FROM customer_journey
GROUP BY derived_persona, churn_risk, upsell_opportunity
ORDER BY customer_count DESC;

In [ ]:
-- 5. 🌍 Global Customer Vibes - How's the love worldwide?
SELECT 
    review_language,
    COUNT(*) as review_count,
    ROUND(AVG(review_rating), 2) as avg_rating,
    ROUND(AVG(sentiment_score), 2) as avg_sentiment,
    COUNT(DISTINCT customer_id) as unique_customers
FROM ANALYTICS.FACT_PRODUCT_REVIEWS
GROUP BY review_language
ORDER BY review_count DESC;

![Streamlit Logo](https://streamlit.io/images/brand/streamlit-logo-primary-colormark-darktext.png) 
# Streamlit Time!


In [ ]:
import streamlit as st
import altair as alt
from snowflake.snowpark.context import get_active_session

# Get Snowflake session
session = get_active_session()

# SQL query
query = """
SELECT 
    priority_level,
    COUNT(*) AS ticket_count,
    ROUND(AVG(sentiment_score), 2) AS avg_sentiment,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM ANALYTICS.FACT_SUPPORT_TICKETS
GROUP BY priority_level
ORDER BY 
    CASE priority_level
        WHEN 'Critical' THEN 1
        WHEN 'High' THEN 2
        WHEN 'Medium' THEN 3
        WHEN 'Low' THEN 4
    END;
"""

# Run query
df = session.sql(query).to_pandas()

# Clean and enforce column types
df.columns = df.columns.str.strip().str.lower()
df['priority_level'] = df['priority_level'].astype(str)
df['ticket_count'] = df['ticket_count'].astype(int)

# Streamlit display
st.title("📊 Support Tickets by Priority Level")

chart = alt.Chart(df).mark_bar().encode(
    x=alt.X("priority_level:N", title="Priority Level", sort=["Critical", "High", "Medium", "Low"]),
    y=alt.Y("ticket_count:Q", title="Number of Tickets"),
    color="priority_level:N"
).properties(width=600)

st.altair_chart(chart, use_container_width=True)
st.dataframe(df)


In [ ]:
import streamlit as st
import altair as alt
from snowflake.snowpark.context import get_active_session

# Get Snowflake session
session = get_active_session()

# SQL query to analyze customer personas
query = """
WITH customer_journey AS (
    SELECT 
        cps.customer_id,
        cps.derived_persona,
        cps.churn_risk,
        cps.upsell_opportunity,
        cps.avg_sentiment,
        cps.sentiment_trend,
        cps.ticket_count,
        cps.avg_rating,
        cps.customer_summary
    FROM ANALYTICS.CUSTOMER_PERSONA_SIGNALS cps
)
SELECT 
    derived_persona,
    churn_risk,
    upsell_opportunity,
    COUNT(*) as customer_count,
    ROUND(AVG(avg_sentiment), 2) as avg_sentiment,
    ROUND(AVG(avg_rating), 2) as avg_rating,
    ROUND(AVG(ticket_count), 2) as avg_tickets
FROM customer_journey
GROUP BY derived_persona, churn_risk, upsell_opportunity
ORDER BY customer_count DESC;
"""

# Run query and convert to pandas DataFrame
df = session.sql(query).to_pandas()

# Clean column names and ensure correct data types
df.columns = df.columns.str.strip().str.lower()
df['churn_risk'] = df['churn_risk'].astype(str)
df['upsell_opportunity'] = df['upsell_opportunity'].astype(str)
df['derived_persona'] = df['derived_persona'].astype(str)

# Streamlit UI
st.title("🧭 Customer Segments by Churn Risk and Upsell Opportunity")

heatmap = alt.Chart(df).mark_rect().encode(
    x=alt.X("churn_risk:N", title="Churn Risk"),
    y=alt.Y("upsell_opportunity:N", title="Upsell Opportunity"),
    color=alt.Color("customer_count:Q", scale=alt.Scale(scheme="blues")),
    tooltip=[
        alt.Tooltip("derived_persona:N", title="Persona"),
        alt.Tooltip("customer_count:Q", title="Customer Count"),
        alt.Tooltip("avg_rating:Q", title="Avg Rating"),
        alt.Tooltip("avg_sentiment:Q", title="Avg Sentiment")
    ]
).properties(width=600, height=400)

st.altair_chart(heatmap, use_container_width=True)
st.dataframe(df)
